Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
Сделайте предсказания на всем df_val. Посчитайте метрику качества.
Дообучите эту модель на df_train.

Посчитайте метрику качества на df_val.

In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00


In [28]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from collections import Counter
from tqdm import tqdm
from transformers import pipeline, BertTokenizer, BertModel

In [4]:
sentiment = pipeline('text-classification', model='sismetanin/RuBERT-ru-sentiment-RuSentiment')
sentiment('Какой ты заботливый друг')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'LABEL_2', 'score': 0.8389314413070679}]

# Dataset loading

In [5]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [6]:
df_train.head(3)

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0


In [7]:
df_test.head(3)

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."


In [8]:
df_val.head(3)

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0


In [10]:
df_train.shape, df_val.shape

((181467, 3), (22683, 3))

Проверим датасет на балансировку классов

In [9]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

Датасет практически сбалансирован.
Проверим модель как она справляется на примере

In [14]:
idx = 2
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT
label is 0


[{'label': 'LABEL_1', 'score': 0.9445229768753052}]

Приведём текст к нижнему регистру

In [15]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [21]:
# bert-base-multilingual-cased

tokenizer = BertTokenizer.from_pretrained('sismetanin/RuBERT-ru-sentiment-RuSentiment')
example_text = "пример текста для токенизации"

bert_input = tokenizer(example_text, padding='max_length', max_length=10,
                       truncation=True, return_tensors='pt')

# input_ids - закодированные id токенов
# attention_mask - чтобы PAD символы не проходили через трансформеры
print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[   101,  22632,  22574,   2748, 111601,  17510,   3426,    102,      0,
              0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [22]:
tokenizer.ids_to_tokens[14337], tokenizer.ids_to_tokens[77572]

('лучших', 'плаванием')

С помощью метода decode можно перейти к человекочетаемому виду

In [23]:
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)

[CLS] пример текста для токенизации [SEP] [PAD] [PAD]


In [24]:
class TwitterDataset(torch.utils.data.Dataset):

  def __init__(self, txts, labels):
      self._labels = labels

      self.tokenizer = BertTokenizer.from_pretrained('sismetanin/RuBERT-ru-sentiment-RuSentiment')
      self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                       truncation=True, return_tensors='pt') for text in txts]

  def __len__(self):
      return len(self._txts)

  def __getitem__(self, index):
      return self._txts[index], self._labels[index]

In [25]:
%%time

y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=2)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=64,
                                           shuffle=False,
                                           num_workers=1)

In [27]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [32]:
class BertClassifier(nn.Module):

    def __init__(self,dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained('sismetanin/RuBERT-ru-sentiment-RuSentiment')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        # _ это набор эмбеддингов слов, а pooled_output - это эмбеддинг предложения
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [35]:
%%time

model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)      # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001) # неполное обучение

CPU times: user 2.29 s, sys: 787 ms, total: 3.08 s
Wall time: 4.74 s


In [36]:
print(model)
print('Parameters full train:', sum([param.nelement() for param in model.parameters()]))
print('Parameters transfer learning:', sum([param.nelement() for param in model.linear.parameters()]))

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [41]:
for epoch_num in range(2):
    total_acc_train, total_loss_train = 0, 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

        print(f'Epoch [{epoch_num + 1} \
          | Train Loss: {total_loss_train / len(train_dataset):.3f}] \
          | Train Accuracy: {total_acc_train / len(train_dataset):.3f} \
          | Val loss: {total_loss_val / len(valid_dataset):.3f} \
          | Val Accuracy: {total_acc_val / len(valid_dataset):.3f}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.002
Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.004
Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.006
Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.007
Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.009
Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.011
Epoch [1           | Train Loss: 0.010]           | Train Accuracy: 0.630           | Val_loss: 0.000           | Val Accuracy: 0.013
Epoch [1           | Train Loss: 0.010]           | Train Accu

 50%|█████     | 1/2 [05:08<05:08, 308.14s/it]

Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.002
Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.004
Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.006
Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.008
Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.009
Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.011
Epoch [2           | Train Loss: 0.010]           | Train Accuracy: 0.629           | Val_loss: 0.000           | Val Accuracy: 0.013
Epoch [2           | Train Loss: 0.010]           | Train Accu

100%|██████████| 2/2 [10:14<00:00, 307.16s/it]
